In [1]:
!pip install catboost osmnx scipy geopy protobuf==3.20.3

In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV, KFold
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from scipy.spatial import cKDTree
from geopy.distance import geodesic
from concurrent.futures import ThreadPoolExecutor
from catboost import CatBoostRegressor, Pool
import osmnx as ox
import geopandas as gpd
from shapely.geometry import Point, MultiPoint
import requests
import json
from tqdm import tqdm
import warnings
warnings.filterwarnings('ignore', message="The indices of the two GeoSeries are different.")

In [3]:
def haversine_distance(lat1, lon1, lat2, lon2):
    R = 6371  # радиус Земли в километрах

    lat1_rad = np.radians(lat1)
    lon1_rad = np.radians(lon1)
    lat2_rad = np.radians(lat2)
    lon2_rad = np.radians(lon2)

    dlat = lat2_rad - lat1_rad
    dlon = lon2_rad - lon1_rad

    a = np.sin(dlat/2)**2 + np.cos(lat1_rad) * np.cos(lat2_rad) * np.sin(dlon/2)**2
    c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1-a))

    distance = R * c
    return distance

In [4]:
df=pd.read_json('train_data.json')
df=pd.concat([df,pd.json_normalize(df['targetAudience'])], axis=1)
df=df.drop(['targetAudience','id'], axis=1)

In [5]:
df['geometry'] = df['points'].apply(lambda x: Point(float(x[0]['lon']), float(x[0]['lat'])))
gdf = gpd.GeoDataFrame(df, geometry='geometry')
gdf.crs = 'EPSG:4326'

In [6]:
moscow = ox.geocode_to_gdf('Moscow, Russia')

In [7]:
# Расстояние до центра (Красная площадь)
red_square = Point(37.620393, 55.753930)
gdf['distance_to_center'] = gdf.apply(lambda row: haversine_distance(row.geometry.y, row.geometry.x, red_square.y, red_square.x), axis=1)

# Ближайшая станция метро
metro_stations = ox.features_from_place('Moscow, Russia', tags={'railway': 'station', 'station': 'subway'})
gdf['distance_to_metro'] = gdf.apply(lambda row: metro_stations.distance(row.geometry).min() / 1000, axis=1)


<ipython-input-7-85064ada1a9d>:7: UserWarning: Geometry is in a geographic CRS. Results from 'distance' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  gdf['distance_to_metro'] = gdf.apply(lambda row: metro_stations.distance(row.geometry).min() / 1000, axis=1)


In [8]:
# Плотность населения (примерные данные)
# population_density = {
#     'Центральный': 5000,
#     'Северный': 3000,
#     'Северо-Восточный': 3500,
#     'Восточный': 4000,
#     'Юго-Восточный': 3800,
#     'Южный': 4200,
#     'Юго-Западный': 3700,
#     'Западный': 3900,
#     'Северо-Западный': 3600,
#     'Зеленоградский': 3200,
#     'Новомосковский': 2500,
#     'Троицкий': 2000
# }

population_density = {
    'Центральный': 11702.67,
    'Северный': 10709.15,
    'Северо-Восточный': 14289.05	,
    'Восточный': 9743.75,
    'Юго-Восточный': 12893.76,
    'Южный': 13422.73,
    'Юго-Западный': 12890.82,
    'Западный': 9312.38,
    'Северо-Западный': 11144.78,
    'Зеленоградский': 7272.25,
    'Новомосковский': 1497.79,
    'Троицкий': 181.13
}

# Функция для определения административного округа
def get_district(point):
    for idx, row in moscow.iterrows():
        if row.geometry.contains(point):
            return row['name']
    return 'Unknown'

gdf['district'] = gdf.apply(lambda row: get_district(row.geometry), axis=1)
gdf['population_density'] = gdf['district'].map(population_density)


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [9]:
shopping_centers = ox.features_from_place('Moscow, Russia', tags={'shop': 'mall'})
gdf['distance_to_shopping_center'] = gdf.apply(lambda row: shopping_centers.distance(row.geometry).min() / 1000, axis=1)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
<ipython-input-9-003a9c48d68e>:2: UserWarning: Geometry is in a geographic CRS. Results from 'distance' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  gdf['distance_to_shopping_center'] = gdf.apply(lambda row: shopping_centers.distance(row.geometry).min() / 1000, axis=1)


In [10]:
def split_on_intervals(min_val, max_val, n):
    step = (max_val - min_val) / n
    intervals = [min_val + (step * x) for x in range(n + 1)]
    return intervals

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [11]:
def create_groups(x_intervals, y_intervals):
    groups = {}
    x_intervals = np.concatenate([[-np.inf], x_intervals, [np.inf]])
    y_intervals = np.concatenate([[-np.inf], y_intervals, [np.inf]])

    for x_i in range(len(x_intervals) - 1):
        for y_i in range(len(y_intervals) - 1):
            groups[f'x:{x_intervals[x_i]:.2f}-{x_intervals[x_i+1]:.2f}|y:{y_intervals[y_i]:.2f}-{y_intervals[y_i+1]:.2f}'] = 0

    return groups

In [12]:
def sort_on_groups(x_vals, y_vals, x_intervals, y_intervals, groups, only_vals=False):
    for x, y in zip(x_vals, y_vals):
        for x_i in range(len(x_intervals) - 1):
            for y_i in range(len(y_intervals) - 1):
                if (x_intervals[x_i] <= x < x_intervals[x_i + 1]) and (y_intervals[y_i] <= y < y_intervals[y_i + 1]):
                    groups[f'x:{x_intervals[x_i]:.2f}-{x_intervals[x_i+1]:.2f}|y:{y_intervals[y_i]:.2f}-{y_intervals[y_i+1]:.2f}'] += 1

    return list(groups.values()) if only_vals else groups

In [13]:
def create_dataset(config, gdf):
    x_intervals = split_on_intervals(config['min_xval'], config['max_xval'], config['x_ngroups'])
    y_intervals = split_on_intervals(config['min_yval'], config['max_yval'], config['y_ngroups'])

    groups = create_groups(x_intervals, y_intervals)

    groups_values = []
    for _, row in gdf.iterrows():
        points = np.array([[float(x['lat']), float(x['lon'])] for x in row['points']])
        group_values = sort_on_groups(points[:, 0], points[:, 1], x_intervals, y_intervals, groups.copy(), only_vals=True)
        groups_values.append(group_values)

    groups_values = np.array(groups_values)

    result = pd.DataFrame(groups_values, columns=list(groups.keys()))

    # Add additional features
    result['num_points'] = gdf['points'].apply(len)
    result['avg_lat'] = gdf['points'].apply(lambda x: np.mean([float(p['lat']) for p in x]))
    result['avg_lon'] = gdf['points'].apply(lambda x: np.mean([float(p['lon']) for p in x]))
    result['avg_azimuth'] = gdf['points'].apply(lambda x: np.mean([p['azimuth'] for p in x]))

    # # Add target audience features
    result['gender'] = gdf['gender'].map({'all': 0, 'male': 1, 'female': 2})
    result['ageFrom'] = gdf['ageFrom']
    result['ageTo'] = gdf['ageTo']
    result['age_range'] = gdf['ageTo'] - gdf['ageFrom']
    result['income'] = gdf['income'].map({'a': 1, 'b': 2, 'c': 3, 'ab': 4, 'bc': 5, 'ac': 6, 'abc': 7})

    #Add new features
    result['distance_to_center'] = gdf['distance_to_center']
    result['distance_to_metro'] = gdf['distance_to_metro']
    result['population_density'] = gdf['population_density']
    result['distance_to_shopping_center'] = gdf['distance_to_shopping_center']

    return result

In [14]:
config = {'min_xval':55.55, 'max_xval':55.95, 'min_yval':37.3, 'max_yval':37.9, 'x_ngroups': 33, 'y_ngroups': 33}

In [15]:
X = create_dataset(config, gdf)
y = df['value']

In [16]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [17]:
model = CatBoostRegressor(iterations=1700,
                          depth=6,
                          learning_rate=0.04,
                          grow_policy='SymmetricTree',
                          random_state=42,
                          loss_function='RMSE')

model.fit(X_train, y_train, eval_set=(X_test, y_test), verbose=100)

0:	learn: 23.7090362	test: 21.7998817	best: 21.7998817 (0)	total: 59.8ms	remaining: 1m 41s
100:	learn: 9.3372228	test: 9.4378079	best: 9.4378079 (100)	total: 1.16s	remaining: 18.4s
200:	learn: 7.4722266	test: 8.1722549	best: 8.1722549 (200)	total: 2.23s	remaining: 16.7s
300:	learn: 6.3170122	test: 7.5529970	best: 7.5529970 (300)	total: 3.32s	remaining: 15.4s
400:	learn: 5.6856184	test: 7.2358616	best: 7.2358616 (400)	total: 4.38s	remaining: 14.2s
500:	learn: 5.1721973	test: 7.0084156	best: 7.0084156 (500)	total: 5.44s	remaining: 13s
600:	learn: 4.8209616	test: 6.8417990	best: 6.8413028 (599)	total: 6.51s	remaining: 11.9s
700:	learn: 4.5891715	test: 6.7861759	best: 6.7837217 (693)	total: 7.7s	remaining: 11s
800:	learn: 4.3496359	test: 6.7172801	best: 6.7172801 (800)	total: 9.64s	remaining: 10.8s
900:	learn: 4.1572546	test: 6.6714219	best: 6.6709926 (899)	total: 11.8s	remaining: 10.5s
1000:	learn: 4.0062527	test: 6.6429842	best: 6.6411867 (994)	total: 14.1s	remaining: 9.83s
1100:	learn: 

In [18]:
y_pred = model.predict(X_test)

In [19]:
rmse = mean_squared_error(y_test, y_pred, squared=False)
r2 = r2_score(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
custom = max(1 - rmse/30, 0) ** 4

print(f'RMSE: {rmse:.4f}')
print(f'R²: {r2:.4f}')
print(f'MAE: {mae:.4f}')
print(f'Custom: {custom:.4f}')

feature_importance = pd.DataFrame({'feature': X.columns, 'importance': model.feature_importances_})
print(feature_importance.sort_values('importance', ascending=False))


RMSE: 6.5932
R²: 0.9112
MAE: 4.2135
Custom: 0.3706
                          feature  importance
1225                   num_points   19.050110
128   x:55.57-55.59|y:37.70-37.72    3.239270
824   x:55.82-55.83|y:37.63-37.65    3.060080
1026  x:55.89-55.90|y:37.48-37.50    2.870601
536   x:55.72-55.73|y:37.48-37.50    2.737688
...                           ...         ...
456   x:55.70-55.71|y:37.30-37.32    0.000000
457   x:55.70-55.71|y:37.32-37.34    0.000000
459   x:55.70-55.71|y:37.35-37.37    0.000000
460   x:55.70-55.71|y:37.37-37.39    0.000000
619   x:55.74-55.76|y:37.72-37.74    0.000000

[1238 rows x 2 columns]


<frozen importlib._bootstrap>:914: ImportWarning: _PyDrive2ImportHook.find_spec() not found; falling back to find_module()
<frozen importlib._bootstrap>:914: ImportWarning: _PyDriveImportHook.find_spec() not found; falling back to find_module()
<frozen importlib._bootstrap>:914: ImportWarning: _GenerativeAIImportHook.find_spec() not found; falling back to find_module()
<frozen importlib._bootstrap>:914: ImportWarning: _OpenCVImportHook.find_spec() not found; falling back to find_module()
<frozen importlib._bootstrap>:914: ImportWarning: APICoreClientInfoImportHook.find_spec() not found; falling back to find_module()
<frozen importlib._bootstrap>:914: ImportWarning: _BokehImportHook.find_spec() not found; falling back to find_module()
<frozen importlib._bootstrap>:914: ImportWarning: _AltairImportHook.find_spec() not found; falling back to find_module()


#Тестирование

In [20]:
def show_predictions(y_test,y_pred):
  for yt, yp in zip(y_test,y_pred):
    print(f'{yt} | {yp}')

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [21]:
slice_idx=10

In [22]:
y_pred = np.round(y_pred, 2)
show_predictions(y_test[:slice_idx],y_pred[:slice_idx])

2.11 | 1.48
33.41 | 17.46
0.0 | -0.61
0.17 | 1.14
53.04 | 56.63
14.13 | 13.2
2.71 | 2.25
17.21 | 13.6
34.89 | 38.02
19.85 | 29.03


In [74]:
from concurrent.futures import ProcessPoolExecutor
from functools import partial

def get_point_coordinates(gdf):
    points = []
    for geom in gdf.geometry:
        if isinstance(geom, Point):
            points.append((geom.x, geom.y))
        elif isinstance(geom, MultiPoint):
            # Для MultiPoint берем первую точку
            points.append((geom.geoms[0].x, geom.geoms[0].y))
    return np.array(points)

# Load and prepare data
metro_stations = ox.features_from_place('Moscow, Russia', tags={'railway': 'station', 'station': 'subway'})
shopping_centers = ox.features_from_place('Moscow, Russia', tags={'shop': 'mall'})
moscow = ox.geocode_to_gdf('Moscow, Russia')

# Function to get centroid coordinates
def get_centroid_coords(geom):
    if geom.geom_type == 'Point':
        return (geom.y, geom.x)
    else:
        centroid = geom.centroid
        return (centroid.y, centroid.x)

# Prepare KD-trees
metro_points = np.array([get_centroid_coords(geom) for geom in metro_stations.geometry])
shopping_points = np.array([get_centroid_coords(geom) for geom in shopping_centers.geometry])
metro_tree = cKDTree(metro_points)
shopping_tree = cKDTree(shopping_points)

# Prepare spatial index
moscow_sindex = moscow.sindex


def enrich_data_vectorized(points, target_audience):
    df = pd.DataFrame(points)
    gdf = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df.lon, df.lat))
    gdf.crs = 'EPSG:4326'

    red_square = (55.753930, 37.620393)

    # Vectorized distance calculation
    coords = np.column_stack((gdf.geometry.y, gdf.geometry.x))
    gdf['distance_to_center'] = geodesic(red_square, coords).kilometers

    # Use KD-tree for faster nearest neighbor search
    gdf['distance_to_metro'] = metro_tree.query(coords)[0]
    gdf['distance_to_shopping_center'] = shopping_tree.query(coords)[0]

    # Vectorized district assignment
    possible_matches_index = list(moscow_sindex.intersection(gdf.total_bounds))
    possible_matches = moscow.iloc[possible_matches_index]
    points_in_districts = gpd.sjoin(gdf, possible_matches, how='left', predicate='within')
    gdf['district'] = points_in_districts['name']

    gdf['population_density'] = gdf['district'].map(population_density)

    # Add target audience data
    for key, value in target_audience.items():
        gdf[key] = value

    return create_dataset(config, gdf)

def predict_reach_batch(points_batch, target_audience):
    X = enrich_data_vectorized(points_batch, target_audience)
    return model.predict(X)


prediction_cache = {}

def predict_reach(points, target_audience):
    # Convert points to a hashable type for caching
    points_key = tuple(sorted((p['lon'], p['lat']) for p in points))
    # Check if we have a cached result
    if points_key in prediction_cache:
        return prediction_cache[points_key]

    # If not in cache, compute the prediction
    df = pd.DataFrame({'points': [points], 'income': [target_audience['income']], 'gender': [target_audience['gender']], 'ageTo': [target_audience['ageTo']], 'ageFrom': [target_audience['ageFrom']] })

    df['geometry'] = df['points'].apply(lambda x: Point(float(x[0]['lon']), float(x[0]['lat'])))
    gdf = gpd.GeoDataFrame(df, geometry='geometry')
    gdf.crs = 'EPSG:4326'

    # Compute features
    red_square = (55.753930, 37.620393)
    gdf['distance_to_center'] = gdf.apply(lambda row: geodesic((row.geometry.y, row.geometry.x), red_square).kilometers, axis=1)

    coords = np.array([(point.y, point.x) for point in gdf.geometry])
    gdf['distance_to_metro'] = metro_tree.query(coords)[0]
    gdf['distance_to_shopping_center'] = shopping_tree.query(coords)[0]

    possible_matches_index = list(moscow_sindex.intersection(gdf.total_bounds))
    possible_matches = moscow.iloc[possible_matches_index]
    points_in_districts = gpd.sjoin(gdf, possible_matches, how='left', predicate='within')
    gdf['district'] = points_in_districts['name']
    gdf['population_density'] = gdf['district'].map(population_density)

    # Add target audience data
    for key, value in target_audience.items():
        gdf[key] = value

    # Create dataset for prediction
    X = create_dataset(config, gdf)

    # Make prediction
    prediction = model.predict(X)[0]

    # Cache the result
    prediction_cache[points_key] = prediction

    return prediction

In [75]:
from tqdm import tqdm

def optimize_campaign_fixed_points(all_points, target_audience, num_points, batch_size=100):
    selected_points = []
    available_points = all_points.copy()
    current_reach = 0

    pbar = tqdm(total=num_points, desc="Selecting points")

    while len(selected_points) < num_points and available_points:
        batch = available_points[:batch_size]
        reach_increases = []
        for point in batch:
            new_reach = predict_reach(selected_points + [point], target_audience)
            reach_increase = new_reach - current_reach
            reach_increases.append(reach_increase)

        best_idx = np.argmax(reach_increases)
        best_increase = reach_increases[best_idx]

        if best_increase > 0:
            best_point = batch[best_idx]
            selected_points.append(best_point)
            available_points.remove(best_point)
            current_reach += best_increase
            pbar.update(1)
        else:
            available_points = available_points[batch_size:]

    pbar.close()
    return selected_points

def chunk_data(data, chunk_size):
    for i in range(0, len(data), chunk_size):
        yield data[i:i + chunk_size]



# def optimize_campaign_variable_points(all_points: list,
#                                       target_audience: dict,
#                                       min_increase_threshold: float) -> list:
#     selected_points = []
#     available_points = all_points.copy()

#     while True:
#         best_point = None
#         best_reach_increase = 0

#         current_reach = predict_reach(selected_points, target_audience)

#         for point in available_points:
#             new_reach = predict_reach(selected_points + [point], target_audience)
#             reach_increase = new_reach - current_reach

#             if reach_increase > best_reach_increase:
#                 best_reach_increase = reach_increase
#                 best_point = point

#         if best_reach_increase > min_increase_threshold:
#             selected_points.append(best_point)
#             available_points.remove(best_point)
#         else:
#             break

#     return selected_points

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [76]:
unique_points = pd.read_json('unique_points.json')

In [77]:
target_audience = {
    "gender": "all",
    "ageFrom": 25,
    "ageTo": 45,
    "income": "bc"
}
all_points = unique_points.to_dict('records')


In [78]:
def run_optimization(all_points, target_audience, num_points, batch_size=100):
    optimal_points = optimize_campaign_fixed_points(all_points, target_audience, num_points, batch_size)
    final_reach = predict_reach(optimal_points, target_audience)
    print(f"Optimal points: {len(optimal_points)}")
    print(f"Predicted reach: {final_reach}")
    return optimal_points, final_reach

In [80]:
optimal_points = run_optimization(all_points, target_audience, num_points=10, batch_size=100)


Selecting points:  10%|█         | 1/10 [02:31<22:43, 151.48s/it]

Selecting points: 100%|██████████| 10/10 [02:42<00:00, 16.22s/it]

Optimal points: 10
Predicted reach: 29.009085810755032


In [105]:
records = pd.read_json('test_data.json')

def calculate_new_column(hash_values, points, target_audience):
    return [f"{hash_value},{max(predict_reach(point, target), 0)}" for hash_value, point, target in zip(hash_values, points, target_audience)]

records['hash,value'] = calculate_new_column(records['hash'], records['points'], records['targetAudience'])
records.drop(['targetAudience', 'points', 'hash'], axis=1, inplace=True)

In [106]:
records['hash,value'].to_csv('submission.csv')

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [108]:
model.save_model('model.cbm')